In [1]:
# Reading data
data = spark.read.csv('/FileStore/tables/SMSSpamCollection',
              inferSchema=True, sep='\t')

In [2]:
# Renaming columns to more meaningful names
data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')

In [3]:
# Showing some data
data.show(10)

+-----+--------------------+
class| text|
+-----+--------------------+
 ham|Go until jurong p...|
 ham|Ok lar... Joking ...|
 spam|Free entry in 2 a...|
 ham|U dun say so earl...|
 ham|Nah I don't think...|
 spam|FreeMsg Hey there...|
 ham|Even my brother i...|
 ham|As per your reque...|
 spam|WINNER!! As a val...|
 spam|Had your mobile 1...|
+-----+--------------------+
only showing top 10 rows

In [4]:
# Importing libraries
from pyspark.sql.functions import length

In [5]:
# Applying length function to text column
data = data.withColumn('length', length(data['text']))

In [6]:
# Showing new dataframe with length column
data.show(10)

+-----+--------------------+------+
class| text|length|
+-----+--------------------+------+
 ham|Go until jurong p...| 111|
 ham|Ok lar... Joking ...| 29|
 spam|Free entry in 2 a...| 155|
 ham|U dun say so earl...| 49|
 ham|Nah I don't think...| 61|
 spam|FreeMsg Hey there...| 147|
 ham|Even my brother i...| 77|
 ham|As per your reque...| 160|
 spam|WINNER!! As a val...| 157|
 spam|Had your mobile 1...| 154|
+-----+--------------------+------+
only showing top 10 rows

In [7]:
# AVG length of SPAM messages is greater than HAM messages
data.groupBy('class').mean().show()

+-----+-----------------+
class| avg(length)|
+-----+-----------------+
 ham|71.45431945307645|
 spam|138.6706827309237|
+-----+-----------------+

In [8]:
# Import NLP tools
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [9]:
# Creating tokenizer object
tokenizer = Tokenizer(inputCol='text', outputCol='tokens')

# Creating stopwords remover object
stopwords = StopWordsRemover(inputCol='tokens', outputCol='sw_tokens')

# Creating count vectorizer object
cv = CountVectorizer(inputCol='sw_tokens', outputCol='count_tokens')

# Creating TF-IDF object
idf = IDF(inputCol='count_tokens', outputCol='tf_idf')

# Creating String indexer object to convert text labels into numeric
ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')

In [10]:
# Importing assembler
from pyspark.ml.feature import VectorAssembler

In [11]:
# Creating assembler
cleanup = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol='features')

In [12]:
# Importing NaiveBayes model
from pyspark.ml.classification import NaiveBayes

In [13]:
# Creating model
nb = NaiveBayes()

In [14]:
# Importing pipelin
from pyspark.ml import Pipeline

In [15]:
# Creating pipeline of steps to execute
pipeline = Pipeline(stages=[
  ham_spam_to_numeric,
  tokenizer,
  stopwords,
  cv,
  idf,
  cleanup
])

In [16]:
# Fitting data into pipeline
data_processed = pipeline.fit(data).transform(data)

In [17]:
# Check the output
data_processed.show(5)

+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
class| text|length|label| tokens| sw_tokens| count_tokens| tf_idf| features|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
 ham|Go until jurong p...| 111| 0.0|[go, until, juron...|[go, jurong, poin...|(13423,[7,11,31,6...|(13423,[7,11,31,6...|(13424,[7,11,31,6...|
 ham|Ok lar... Joking ...| 29| 0.0|[ok, lar..., joki...|[ok, lar..., joki...|(13423,[0,24,297,...|(13423,[0,24,297,...|(13424,[0,24,297,...|
 spam|Free entry in 2 a...| 155| 1.0|[free, entry, in,...|[free, entry, 2, ...|(13423,[2,13,19,3...|(13423,[2,13,19,3...|(13424,[2,13,19,3...|
 ham|U dun say so earl...| 49| 0.0|[u, dun, say, so,...|[u, dun, say, ear...|(13423,[0,70,80,1...|(13423,[0,70,80,1...|(13424,[0,70,80,1...|
 ham|Nah I don't think...| 61| 0.0|[nah, i, don't, t...|[nah, think, goes...|(13423,[36,134,31...|(13423,[36,134,31...|(13424,[36,134,31...|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows

In [18]:
# Filtering out only needed columns
data_processed = data_processed.select(['label', 'features'])

In [19]:
# Showing final data
data_processed.show(5)

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(13424,[7,11,31,6...|
 0.0|(13424,[0,24,297,...|
 1.0|(13424,[2,13,19,3...|
 0.0|(13424,[0,70,80,1...|
 0.0|(13424,[36,134,31...|
+-----+--------------------+
only showing top 5 rows

In [20]:
# Splitting data
(train_set, test_set) = data_processed.randomSplit([0.8, 0.2])

In [21]:
# Training model with train data
spam_model = nb.fit(train_set)

In [22]:
# Predicting SPAM x HAM
spam_ham_preds = spam_model.transform(test_set)

In [23]:
spam_ham_preds.show(5)

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(13424,[0,1,5,15,...|[-1004.9659595926...|[1.0,2.7580092201...| 0.0|
 0.0|(13424,[0,1,7,8,1...|[-885.73287286439...|[1.0,9.3179713126...| 0.0|
 0.0|(13424,[0,1,9,14,...|[-540.96976471526...|[1.0,1.7447996452...| 0.0|
 0.0|(13424,[0,1,12,33...|[-441.87891554736...|[1.0,1.2079367944...| 0.0|
 0.0|(13424,[0,1,14,18...|[-1384.7034643299...|[1.0,4.1912110697...| 0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows

In [24]:
# Importing evaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [25]:
# Evaluating model
accuracy = MulticlassClassificationEvaluator().evaluate(spam_ham_preds)

In [26]:
print("Spam detector's accuracy: {}".format(accuracy))

Spam detector's accuracy: 0.922979333131